In [5]:
#Dependancies
import pandas as pd
from sqlalchemy import *
from sqlalchemy.schema import CreateSchema
import pymysql
from config import username,pwd,host

In [6]:
#importing csv file
csvfile="Census_CSV_data.csv"
df=pd.read_csv(csvfile)

/anaconda3/envs/PythonEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,CENSUS_YEAR,GEO_CODE (POR),GEO_LEVEL,GEO_NAME,GNR,GNR_LF,DATA_QUALITY_FLAG,ALT_GEO_CODE,DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Member ID: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Notes: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
0,2016,1,0,Canada,4.0,5.1,20000,1,"Population, 2016",1,1.0,3.51517e+07,...,...
1,2016,1,0,Canada,4.0,5.1,20000,1,"Population, 2011",2,2.0,3.34767e+07,...,...
2,2016,1,0,Canada,4.0,5.1,20000,1,"Population percentage change, 2011 to 2016",3,NaN,5,...,...
3,2016,1,0,Canada,4.0,5.1,20000,1,Total private dwellings,4,3.0,1.54124e+07,...,...
4,2016,1,0,Canada,4.0,5.1,20000,1,Private dwellings occupied by usual residents,5,4.0,1.40721e+07,...,...


In [8]:
#rows*columns
df.shape

(790944, 14)

In [9]:
#column names
df.columns

Index(['CENSUS_YEAR', 'GEO_CODE (POR)', 'GEO_LEVEL', 'GEO_NAME', 'GNR',
       'GNR_LF', 'DATA_QUALITY_FLAG', 'ALT_GEO_CODE',
       'DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)',
       'Member ID: Profile of Federal Electoral Districts (2013 Representation Order) (2247)',
       'Notes: Profile of Federal Electoral Districts (2013 Representation Order) (2247)',
       'Dim: Sex (3): Member ID: [1]: Total - Sex',
       'Dim: Sex (3): Member ID: [2]: Male',
       'Dim: Sex (3): Member ID: [3]: Female'],
      dtype='object')

In [10]:
#Selecting required columns
df=df[['CENSUS_YEAR','GEO_LEVEL','GEO_NAME','DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)','Dim: Sex (3): Member ID: [1]: Total - Sex','Dim: Sex (3): Member ID: [2]: Male','Dim: Sex (3): Member ID: [3]: Female']]
      

In [11]:
df.head()

,CENSUS_YEAR,GEO_LEVEL,GEO_NAME,DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
0,2016,0,Canada,"Population, 2016",3.51517e+07,...,...
1,2016,0,Canada,"Population, 2011",3.34767e+07,...,...
2,2016,0,Canada,"Population percentage change, 2011 to 2016",5,...,...
3,2016,0,Canada,Total private dwellings,1.54124e+07,...,...
4,2016,0,Canada,Private dwellings occupied by usual residents,1.40721e+07,...,...


In [12]:
df.shape

(790944, 7)

In [20]:
# Renaming columns
df=df.rename(columns = {'DIM: Profile of Federal Electoral Districts (2013 Representation Order) (2247)': "Metric",'Dim: Sex (3): Member ID: [1]: Total - Sex':"Total",'Dim: Sex (3): Member ID: [2]: Male': 'Male','Dim: Sex (3): Member ID: [3]: Female': 'Female'})
df=df.rename(columns = {'CENSUS_YEAR':'Census_year','GEO_LEVEL':'Geo_level','GEO_NAME':'Geo_name'})
df.head()

,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female
0,2016,0,Canada,"Population, 2016",3.51517e+07,...,...
1,2016,0,Canada,"Population, 2011",3.34767e+07,...,...
2,2016,0,Canada,"Population percentage change, 2011 to 2016",5,...,...
3,2016,0,Canada,Total private dwellings,1.54124e+07,...,...
4,2016,0,Canada,Private dwellings occupied by usual residents,1.40721e+07,...,...


In [21]:
df.shape

(790944, 7)

In [28]:
df_population_2016=df.loc[df["Metric"]=="Population, 2016", :]
df_population_2016.insert(0,'Id',range(1,1+len(df_population_2016)))
df_population_2016.head()

,Id,Census_year,Geo_level,Geo_name,Metric,Total,Male,Female
0,1,2016,0,Canada,"Population, 2016",3.51517e+07,...,...
2247,2,2016,1,Newfoundland and Labrador,"Population, 2016",519716,...,...
4494,3,2016,2,Avalon,"Population, 2016",86494,...,...
6741,4,2016,2,Bonavista--Burin--Trinity,"Population, 2016",74116,...,...
8988,5,2016,2,Coast of Bays--Central--Notre Dame,"Population, 2016",77680,...,...


In [23]:
df_population_2016.shape

(352, 7)

In [17]:
#database connection
metadata = MetaData()
engine = create_engine(f'mysql+pymysql://{username}:{pwd}@{host}:3306')
engine.execute(CreateSchema('ValueofVote'))

In [29]:
engine.execute("USE ValueofVote")
metadata = MetaData()

In [30]:
#create table
Population_2016 = Table('population_2016', metadata,
    Column('Id', Integer, primary_key=True),
    Column('Census_year', Integer, nullable=False),
    Column('Geo_level', Integer, nullable=False),
    Column('Geo_Name', String(100), nullable=False),
    Column('Metric', String(100), nullable=False),
    Column('Total', Integer, nullable=True),
    Column('Male', Integer, nullable=True),
    Column('Female', Integer, nullable=True)
)

In [32]:
df_population_2016.to_sql(name='population_2016', con=engine, if_exists='replace', index=False)